<a href="https://colab.research.google.com/github/matt-needle/project-data/blob/master/docs/examples/train_object_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train an Object Detection Model with GeoAI

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/train_object_detection_model.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [1]:
%pip install geoai-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.8/153.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.1/568.1 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.7/286.7 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.

## Import libraries

In [2]:
import os
import geoai

out_folder = "output"

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


## Download sample data

## Train object detection model

In [7]:
import os

from google.colab import drive
drive.mount('/content/drive')

out_folder = "trial"

# print('images:', len(os.listdir('/content/drive/MyDrive/geoai/' + out_folder + '/images')), 'labels:', len(os.listdir('/content/drive/MyDrive/geoai/' + out_folder + '/labels')))

# Access your data
# !cp /content/drive/MyDrive/geoai/output .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# !cp /content/drive/MyDrive/geoai/output ./output -r

!cp /content/drive/MyDrive/geoai/trial ./output -r

In [6]:
out_folder = '/content/output/'
# out_folder = '/content/drive/MyDrive/geoai/output'

geoai.train_MaskRCNN_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/models",
    num_channels=4,
    pretrained=True,
    batch_size=12,
    num_epochs=100,
    learning_rate=0.001,
    val_split=0.1,
)

Using device: cuda
Found 948 image files and 12 label files
Using 12 matching files
Training on 10 images, validating on 2 images


Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:01<00:00, 158MB/s]


Epoch: 0, Batch: 0/1, Loss: 3.9161, Time: 5.92s
Epoch 1/100: Train Loss: 3.9161, Val Loss: inf, Val IoU: 0.0149
Saving best model with IoU: 0.0149
Epoch: 1, Batch: 0/1, Loss: 2.7405, Time: 2.94s
Epoch 2/100: Train Loss: 2.7405, Val Loss: inf, Val IoU: 0.0160
Saving best model with IoU: 0.0160
Epoch: 2, Batch: 0/1, Loss: 2.1231, Time: 3.03s
Epoch 3/100: Train Loss: 2.1231, Val Loss: inf, Val IoU: 0.0235
Saving best model with IoU: 0.0235
Epoch: 3, Batch: 0/1, Loss: 1.9275, Time: 3.37s
Epoch 4/100: Train Loss: 1.9275, Val Loss: inf, Val IoU: 0.0300
Saving best model with IoU: 0.0300
Epoch: 4, Batch: 0/1, Loss: 1.7861, Time: 2.92s
Epoch 5/100: Train Loss: 1.7861, Val Loss: inf, Val IoU: 0.0310
Saving best model with IoU: 0.0310
Epoch: 5, Batch: 0/1, Loss: 1.6830, Time: 2.96s
Epoch 6/100: Train Loss: 1.6830, Val Loss: inf, Val IoU: 0.0381
Saving best model with IoU: 0.0381
Epoch: 6, Batch: 0/1, Loss: 1.6328, Time: 3.08s
Epoch 7/100: Train Loss: 1.6328, Val Loss: inf, Val IoU: 0.0446
Saving

RasterioIOError: Caught RasterioIOError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "rasterio/_base.pyx", line 310, in rasterio._base.DatasetBase.__init__
  File "rasterio/_base.pyx", line 221, in rasterio._base.open_dataset
  File "rasterio/_err.pyx", line 359, in rasterio._err.exc_wrap_pointer
rasterio._err.CPLE_OpenFailedError: /content/drive/MyDrive/geoai/output/labels/tile_546.tif: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
images_dir=f"{out_folder}/images"

os.listdir(images_dir)

## Run inference

In [ ]:
masks_path = "naip_test_prediction.tif"
model_path = f"{out_folder}/models/best_model.pth"

In [ ]:
geoai.object_detection(
    test_raster_path,
    masks_path,
    model_path,
    window_size=512,
    overlap=256,
    confidence_threshold=0.5,
    batch_size=4,
    num_channels=4,
)

## Vectorize masks

In [ ]:
output_path = "naip_test_prediction.geojson"
gdf = geoai.orthogonalize(masks_path, output_path, epsilon=2)

## Visualize results

In [ ]:
geoai.view_vector_interactive(output_path, tiles=test_raster_url)

In [ ]:
geoai.create_split_map(
    left_layer=output_path,
    right_layer=test_raster_url,
    left_args={"style": {"color": "red", "fillOpacity": 0.2}},
    basemap=test_raster_url,
)

![image](https://github.com/user-attachments/assets/8dfcc69e-7a6c-408a-9fae-10b81b7d85dc)